Install the `openai` package along with its required dependencies.

In [1]:
%pip install --upgrade "openai>=1.95" "pydantic-core>=2.16.3" "pydantic>=2.6" 
%pip install --upgrade "attrs"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.4/764.4 kB 19.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 8.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 1.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 1.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 1.5 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.5/352.5 kB 8.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 2.4 MB/s eta 0:00:00
  

Locate the paths to the three installed packages.

In [2]:
%pip show openai pydantic-core pydantic attrs  

Name: openai
Version: 1.97.1
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: /cloud/lib/lib/python3.10/site-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 
---
Name: pydantic_core
Version: 2.33.2
Summary: Core functionality for Pydantic validation and serialization
Home-page: https://github.com/pydantic/pydantic-core
Author: 
Author-email: Samuel Colvin <s@muelcolvin.com>, Adrian Garcia Badaracco <1755071+adriangb@users.noreply.github.com>, David Montague <david@pydantic.dev>, David Hewitt <mail@davidhewitt.dev>, Sydney Runkle <sydneymarierunkle@gmail.com>, Victorien Plot <contact@vctrn.dev>
License: MIT
Location: /cloud/lib/lib/python3.10/site-packages
Requires: typing-extensions
Required-by: pydantic
---
Name: pydantic
Version: 2.11.7
Summary: Data validation using Python type hints
Home-page: 
Author: 
Author-email: Samuel Colv

Add the path to `sys.path`; then list the full contents of `sys.path`, which can help diagnose missing modules or packages.

In [3]:
import sys
sys.path.append("/cloud/lib/lib/python3.10/site-packages")
print(sys.path)

['/opt/python/3.10.12/lib/python310.zip', '/opt/python/3.10.12/lib/python3.10', '/opt/python/3.10.12/lib/python3.10/lib-dynload', '', '/opt/python/3.10.12/lib/python3.10/site-packages', '/cloud/lib/lib/python3.10/site-packages']


Retrieve your OpenAI API key from your account dashboard at https://platform.openai.com/. If you don’t already have a key, generate one there.

In [1]:
# Replace "your_api_key_here" with your actual API key
api_key ="your_api_key_here"

Convert the csv data file into a jsonl file for training purposes

In [5]:
import csv
import json

INPUT_CSV  = "qna_chitchat_friendly.csv"
OUTPUT_JL  = "training_data.jsonl"

SYSTEM_PROMPT = "You are a friendly assistant."

with open(INPUT_CSV, newline="", encoding="utf-8") as csvfile, \
     open(OUTPUT_JL,  "w",      encoding="utf-8") as jsonlfile:

    reader = csv.DictReader(csvfile)
    for row in reader:
        example = {
            "messages": [
                {"role": "system",    "content": SYSTEM_PROMPT},
                {"role": "user",      "content": row["Question"].strip()},
                {"role": "assistant", "content": row["Answer"].strip()}
            ]
        }
        jsonlfile.write(json.dumps(example, ensure_ascii=False) + "\n")

print("Wrote chat‑formatted JSONL to", OUTPUT_JL)

Wrote chat‑formatted JSONL to training_data.jsonl


Upload the `training_data.jsonl` file to OpenAI and obtain the assigned file id.

In [6]:
from openai import OpenAI          
client = OpenAI(api_key=api_key)   

with open("training_data.jsonl", "rb") as file:
    upload_response = client.files.create(   
        file=file,
        purpose="fine-tune"
    )

file_id = upload_response.id
print("Uploaded file id:", file_id)


Uploaded file id: file-2FSJTmDHLiDgLZf8iCqy6Y


Because OpenAI’s o3 and o4‑mini models don’t currently support fine‑tuning, we trained the sample data on the closest available option `gpt-4o-mini-2024-07-18`.  

In [7]:
job = client.fine_tuning.jobs.create(
    training_file="file-2FSJTmDHLiDgLZf8iCqy6Y",
    model="gpt-4o-mini-2024-07-18",            
    suffix="my-4o-mini",
    hyperparameters={"n_epochs": 4}
)
print("Job id:", job.id, "| status:", job.status)

Job id: ftjob-QR9AebCZHDBtGXej0UWq64iz | status: validating_files


You may keep checking the job status, but allow 30 minutes or longer for OpenAI to build the fine-tuned model, keeping in mind that the time required may vary depending on the size of the model being fine-tuned. After the model is successfully built, go to https://platform.openai.com/finetune/ to reterive the model ID the model ID as below.

![Fine-tuned model id](https://filedn.com/lJpzjOtA91quQEpwdrgCvcy/Business%20Data%20Mining%20and%20Knowledge%20Discovery/Module%203/Fine-tuned%20model%20id%20update.png)

In [4]:
from openai import OpenAI          
client = OpenAI(api_key=api_key)  

# replace ***** with the actual code in your model id 
MODEL_ID = "ft:gpt-4o-mini-2024-07-18:personal:my-4o-mini:*****"

prompt = "What is your name?"

chat_response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.8,   # Controls the randomness and creativity of the generated output. 
                           # Lower values will produce more conservative output, while higher 
                           # values will produce more creative output. The default value is 1.0, 
                           # and 0.8 is used in this example.
    
        max_tokens=30,     # The maximum number of tokens (words or sub-words) in the generated output. 
                           # If the model reaches this limit, it will stop generating new output. 
                           # 30 is used in this example.
    
        timeout=30,        # The maximum time in seconds to wait for a response from the OpenAI API. If the API does not 
                           # respond within this time, an exception will be raised. The default value is 60 seconds, and 
                           # 30 is used in this example.
    )
print(chat_response.choices[0].message.content)

Oh, I don't have a name.


In [5]:
prompt = "where are you from?"

chat_response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.8,   # Controls the randomness and creativity of the generated output. 
                           # Lower values will produce more conservative output, while higher 
                           # values will produce more creative output. The default value is 1.0, 
                           # and 0.8 is used in this example.
    
        max_tokens=30,     # The maximum number of tokens (words or sub-words) in the generated output. 
                           # If the model reaches this limit, it will stop generating new output. 
                           # 30 is used in this example.
    
        timeout=30,        # The maximum time in seconds to wait for a response from the OpenAI API. If the API does not 
                           # respond within this time, an exception will be raised. The default value is 60 seconds, and 
                           # 30 is used in this example.
    )
print(chat_response.choices[0].message.content)

I'm digital, so I'm always just... here.


In [6]:
prompt = "I really like you!"

chat_response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.8,   # Controls the randomness and creativity of the generated output. 
                           # Lower values will produce more conservative output, while higher 
                           # values will produce more creative output. The default value is 1.0, 
                           # and 0.8 is used in this example.
    
        max_tokens=30,     # The maximum number of tokens (words or sub-words) in the generated output. 
                           # If the model reaches this limit, it will stop generating new output. 
                           # 30 is used in this example.
    
        timeout=30,        # The maximum time in seconds to wait for a response from the OpenAI API. If the API does not 
                           # respond within this time, an exception will be raised. The default value is 60 seconds, and 
                           # 30 is used in this example.
    )
print(chat_response.choices[0].message.content)

Thanks! I like you too!


In [7]:
prompt = "where should we go for dinner?"

chat_response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.8,   # Controls the randomness and creativity of the generated output. 
                           # Lower values will produce more conservative output, while higher 
                           # values will produce more creative output. The default value is 1.0, 
                           # and 0.8 is used in this example.
    
        max_tokens=30,     # The maximum number of tokens (words or sub-words) in the generated output. 
                           # If the model reaches this limit, it will stop generating new output. 
                           # 30 is used in this example.
    
        timeout=30,        # The maximum time in seconds to wait for a response from the OpenAI API. If the API does not 
                           # respond within this time, an exception will be raised. The default value is 60 seconds, and 
                           # 30 is used in this example.
    )
print(chat_response.choices[0].message.content)

I think you should follow your heart.


In [9]:
prompt = "who will win 2028 presidental election?"

chat_response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.8,   # Controls the randomness and creativity of the generated output. 
                           # Lower values will produce more conservative output, while higher 
                           # values will produce more creative output. The default value is 1.0, 
                           # and 0.8 is used in this example.
    
        max_tokens=30,     # The maximum number of tokens (words or sub-words) in the generated output. 
                           # If the model reaches this limit, it will stop generating new output. 
                           # 30 is used in this example.
    
        timeout=30,        # The maximum time in seconds to wait for a response from the OpenAI API. If the API does not 
                           # respond within this time, an exception will be raised. The default value is 60 seconds, and 
                           # 30 is used in this example.
    )
print(chat_response.choices[0].message.content)

I don't know.


In [10]:
prompt = "why Maine the best state in the US?"

chat_response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.8,   # Controls the randomness and creativity of the generated output. 
                           # Lower values will produce more conservative output, while higher 
                           # values will produce more creative output. The default value is 1.0, 
                           # and 0.8 is used in this example.
    
        max_tokens=30,     # The maximum number of tokens (words or sub-words) in the generated output. 
                           # If the model reaches this limit, it will stop generating new output. 
                           # 30 is used in this example.
    
        timeout=30,        # The maximum time in seconds to wait for a response from the OpenAI API. If the API does not 
                           # respond within this time, an exception will be raised. The default value is 60 seconds, and 
                           # 30 is used in this example.
    )
print(chat_response.choices[0].message.content)

With questions like this, I'm not much better than a Ouija board.


In [13]:
prompt = "will AI change the world?"

chat_response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.8,   # Controls the randomness and creativity of the generated output. 
                           # Lower values will produce more conservative output, while higher 
                           # values will produce more creative output. The default value is 1.0, 
                           # and 0.8 is used in this example.
    
        max_tokens=30,     # The maximum number of tokens (words or sub-words) in the generated output. 
                           # If the model reaches this limit, it will stop generating new output. 
                           # 30 is used in this example.
    
        timeout=30,        # The maximum time in seconds to wait for a response from the OpenAI API. If the API does not 
                           # respond within this time, an exception will be raised. The default value is 60 seconds, and 
                           # 30 is used in this example.
    )
print(chat_response.choices[0].message.content)

The impact of AI on society will depend on how we choose to use it.
